In [13]:
import requests
import wget

In [14]:
download_root = "https://www.kaggle.com/corrieaar/apartment-rental-offers-in-germany/download"
#HOUSING_PATH = os.path.join("datasets", "housing")
#HOUSING_URL = download_root + "datasets/housing/housing.tgz"
wget.download(download_root, "/home/Sina/ApartmentRentalOffersInBerlin/dataset/apartment-rental-offers-in-germany.zip")

NameError: name 'wget' is not defined